In [70]:
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_compression as tfc
import re
import random
import numpy as np
import os
from astropy.io import fits

# Specify the directory containing the .tar files
directory_path = './columbialensing/'



# image_size
image_size = 1024
number_image_files = 512
suffix = f"_{image_size}"
extract_tarfiles = False  #if I need to extract tarfiles

#number of k bins
kbins = image_size  #current this has to be equal to work


run_suffix = rf"im{image_size}"



def get_labels_for_file(dir_name):
    """
    Extracts labels from the tar file name.
    For the file "Om0.183_si0.958_256.tar", the labels will be [0.183, 0.958].

    Args:
    - tar_file_name (str): Name of the tar file.

    Returns:
    - list: List containing the two labels extracted from the filename.
    """
    # Split the filename on underscores
    if dir_name.endswith('.tar'):
        dir_name = dir_name[:-4]

    #print(dir_name)
    parts = dir_name.split('_')

    # Extract the numeric values for 'Om' and 'si'
    om_label = float(parts[0][2:])
    si_label = float(parts[1][2:])
    #print (om_label, si_label)

    return [om_label, si_label]

#selects the numdirs closest to the fiducial directory
def select_some_directories(numdirs, fiducial_dir):
    all_directories = [f for f in os.listdir(directory_path) if '128' not in f and '256' not in f and 'Om' in f and '.tar' not in f]
    #print(all_directories)
    #cosmology_labels = np.empty((len(all_directories), number_images, 2), dtype=np.float16)
    fiducial_label = get_labels_for_file(fiducial_dir)
    labels = np.array([get_labels_for_file(directory) for directory in all_directories])
    #print(labels[:,0])
    #to evlauate closeness
    closeness = pow(labels[:,0]/.3, 0.5)*labels[:,1]
    fiducial = pow(fiducial_label[0]/.3, 0.5)*fiducial_label[1]

    #sort by closeness
    sorted_closeness = np.argsort(np.abs(closeness-fiducial))
    #select numdirs closest
    selected_dirs = sorted_closeness[:numdirs]
    return [all_directories[i] for i in selected_dirs]



#now loop through all files in the
#pattern = re.compile(rf"{suffix}$")



In [71]:
numdirs=10
all_directories = select_some_directories(numdirs, "Om0.268_si0.801")
print(all_directories)




['Om0.268_si0.801', 'Om0.250_si0.825', 'Om0.274_si0.786', 'Om0.234_si0.864', 'Om0.258_si0.823', 'Om0.291_si0.775', 'Om0.315_si0.746', 'Om0.259_si0.806', 'Om0.183_si0.958', 'Om0.261_si0.802']


In [69]:
import os
import tarfile

for tar_file in all_directories[1:]:
    print(tar_file)
    tar_file_path = os.path.join(directory_path, tar_file)

    # Extract the tar archive
    with tarfile.open(tar_file_path, 'r') as archive:
        archive.extractall(path=directory_path)

    dir_name = os.path.splitext(tar_file)[0]
    
    dir_path = os.path.join(directory_path, dir_name)

    #all_files = os.listdir(dir_path)
    #fits_files = [f for f in all_files if f.endswith('.fits')]

    # After processing the files, create a tar file from the directory
    #with tarfile.open(tar_file_path, 'w') as archive:
    #archive.add(dir_path, arcname=os.path.basename(dir_path))
    #shutil.rmtree(dir_path)

Om0.250_si0.825.tar
Om0.274_si0.786.tar
Om0.234_si0.864.tar
Om0.258_si0.823.tar
Om0.291_si0.775.tar
Om0.315_si0.746.tar
Om0.259_si0.806.tar
Om0.183_si0.958.tar
Om0.261_si0.802.tar


# Fourier transform functions

In [74]:
def fft_radial_distance(image_size):
    # Create indices for the Fourier space
    kx, ky = np.meshgrid(np.fft.rfftfreq(image_size ), np.fft.fftfreq(image_size ))
    k = np.sqrt(kx**2 + ky**2)*image_size  # Calculate radial distance in Fourier space
    return k

def radial_bin_average(image_fft, radial_dist):
    # Calculate the magnitude of the FFT (power spectrum)
    magnitude = np.abs(image_fft)**2

    # Bin the values based on radial distance
    radial_dist_int = np.rint(radial_dist).astype(int)
    tbin = np.bincount(radial_dist_int.ravel(), magnitude.ravel())
    nr = np.bincount(radial_dist_int.ravel())
    #print(tbin, nr)
    radial_profile = tbin / nr
    return radial_profile

def radial_bin_average_cross(image_fft1, image_fft2, radial_dist):
    # Calculate the magnitude of the FFT (power spectrum)
    magnitude = np.abs(image_fft1*np.conj(image_fft2))

    # Bin the values based on radial distance
    radial_dist_int = np.rint(radial_dist).astype(int)
    tbin = np.bincount(radial_dist_int.ravel(), magnitude.ravel())
    nr = np.bincount(radial_dist_int.ravel())
    #print(tbin, nr)
    radial_profile = tbin / nr
    return radial_profile

def average_fft_power_spectrum(fftimages, shape):
    num_images = len(fftimages)
    radial_dist = fft_radial_distance(shape)
    
    # Calculate the maximum bin index, rounding up to the nearest integer
    max_bin_index = int(np.ceil(np.max(radial_dist)))
    #print(max_bin_index, radial_dist)
    
    # Initialize the sum of radial power spectra
    sum_radial_power_spectrum = np.zeros(max_bin_index)
    
    for fftimage in fftimages:
        #fftimage = np.fft.fftshift(fftimage)
        radial_power_spectrum = radial_bin_average(fftimage, radial_dist)
        sum_radial_power_spectrum += radial_power_spectrum
    
    # Average across all images
    avg_radial_power_spectrum = sum_radial_power_spectrum / num_images
    return avg_radial_power_spectrum


def average_fft_cross_power_spectrum(fftimages1, fftimages2, shape):
    num_images1 = len(fftimages1); num_images2 = len(fftimages2)

    if(num_images1 != num_images2):
        print("number of images in each set must be the same:", num_images1, num_images2, "Aborting")
        return
    
    radial_dist = fft_radial_distance(shape)
    
    # Calculate the maximum bin index, rounding up to the nearest integer
    max_bin_index = int(np.ceil(np.max(radial_dist)))
    
    # Initialize the sum of radial power spectra
    sum_radial_power_spectrum = np.zeros(max_bin_index)

    for fftimage1, fftimage2 in zip(fftimages1, fftimages2):
        #fftimage = np.fft.fftshift(fftimage)
        radial_power_spectrum = radial_bin_average_cross(fftimage1, fftimage2, radial_dist)
        sum_radial_power_spectrum += radial_power_spectrum
    
    # Average across all images
    avg_radial_power_spectrum = sum_radial_power_spectrum / num_images1
    return avg_radial_power_spectrum

In [81]:


nmax = 10 #maximum number of files (set to -1 to use all of them)


#num_cosmologies = len(all_directories)



RMS =0 #first time set to zero

power_matrix = np.zeros([kbins, numdirs, numdirs])

for idirectory, dir_name1 in enumerate(all_directories):
    dir_path1 = os.path.join(directory_path, dir_name1)
    print("1 ", dir_path1 )
    all_files1 = os.listdir(dir_path1)
    fits_files1 = [f for f in all_files1 if f.endswith('.fits')]
    for jdirectory, dir_name2 in enumerate(all_directories):
        dir_path2 = os.path.join(directory_path, dir_name2)
        print("2", dir_path2)
        all_files2 = os.listdir(dir_path2)
        fits_files2 = [f for f in all_files2 if f.endswith('.fits')]

        fft1 = np.zeros((nmax, image_size, image_size//2+1), dtype=np.complex64)
        fft2 = np.zeros((nmax, image_size, image_size//2+1), dtype=np.complex64)
        for i, file1 in enumerate(fits_files1[:nmax]):
            with fits.open(os.path.join(dir_path1, file1)) as hdul1:
                data1 = hdul1[0].data
                fft1[i] = np.fft.rfft2(data1)
        for j, file2 in enumerate(fits_files2[:nmax]):
            with fits.open(os.path.join(dir_path2, file2)) as hdul2:
                data2 = hdul2[0].data
                fft2[j] = np.fft.rfft2(data2)

        Pk1 = average_fft_cross_power_spectrum(fft1, fft1, kbins)
        Pk2 = average_fft_cross_power_spectrum(fft2, fft2, kbins)
        cross = average_fft_cross_power_spectrum(fft1, fft2, kbins)
        print(len(cross), len(Pk1), len(Pk2))   
        power_matrix[:len(cross), idirectory, jdirectory] += cross/np.sqrt(Pk1*Pk2)

    if nmax !=-1:
        power_matrix[:, idirectory, jdirectory] /= nmax**2
    else:
        power_matrix[:, idirectory, jdirectory] /= len(fits_files1)**2        
                    

    

1  ./columbialensing/Om0.268_si0.801
2 ./columbialensing/Om0.268_si0.801
725 725 725
2 ./columbialensing/Om0.250_si0.825
725 725 725
2 ./columbialensing/Om0.274_si0.786
725 725 725
2 ./columbialensing/Om0.234_si0.864
725 725 725
2 ./columbialensing/Om0.258_si0.823
725 725 725
2 ./columbialensing/Om0.291_si0.775
725 725 725
2 ./columbialensing/Om0.315_si0.746
725 725 725
2 ./columbialensing/Om0.259_si0.806
725 725 725
2 ./columbialensing/Om0.183_si0.958
725 725 725
2 ./columbialensing/Om0.261_si0.802
725 725 725
1  ./columbialensing/Om0.250_si0.825
2 ./columbialensing/Om0.268_si0.801
725 725 725
2 ./columbialensing/Om0.250_si0.825
725 725 725
2 ./columbialensing/Om0.274_si0.786
725 725 725
2 ./columbialensing/Om0.234_si0.864
725 725 725
2 ./columbialensing/Om0.258_si0.823
725 725 725
2 ./columbialensing/Om0.291_si0.775
725 725 725
2 ./columbialensing/Om0.315_si0.746
725 725 725
2 ./columbialensing/Om0.259_si0.806
725 725 725
2 ./columbialensing/Om0.183_si0.958
725 725 725
2 ./columbiale

In [93]:
epsilon=1e-300
kminus = 300
determinant = np.array([np.linalg.det(power_matrix[i, :, :]) for i in range(kbins)])
bitsperpix = np.log2(determinant[:-kminus]+epsilon)/(2*len(all_directories))

print("det = ", determinant)
print("bits per pixel = ", bitsperpix)

det =  [4.92205292e-38 2.11549190e-32 3.37023927e-29 ... 0.00000000e+00
 0.00000000e+00 0.00000000e+00]
bits per pixel =  [-6.19670037 -5.26103529 -4.72915319 -4.09987214 -3.83039471 -3.5004172
 -3.30296735 -3.10720902 -3.07909546 -2.94958958 -2.82180539 -2.70506625
 -2.67884416 -2.56053137 -2.49496785 -2.4777085  -2.39071034 -2.33113671
 -2.25888457 -2.23974872 -2.18871697 -2.17984056 -2.1027848  -2.10634917
 -2.06957353 -2.01782703 -2.04476972 -2.00202846 -2.00104684 -1.92551115
 -1.92043313 -1.91194401 -1.87636981 -1.83824535 -1.86042239 -1.83754451
 -1.78824025 -1.79714662 -1.77024637 -1.74222568 -1.73836793 -1.73647786
 -1.73332047 -1.68438171 -1.6885804  -1.65863943 -1.66298828 -1.64467363
 -1.64369988 -1.64368715 -1.61729948 -1.61460726 -1.60124001 -1.59639945
 -1.58452071 -1.57417187 -1.55528095 -1.55341074 -1.54123713 -1.54376856
 -1.54467884 -1.5169319  -1.5163354  -1.51234559 -1.50770555 -1.49365414
 -1.4808384  -1.48149274 -1.48098175 -1.47691773 -1.46863818 -1.46089182
 -1

In [116]:
#[power_matrix[-500, n, n+2] for n in range(numdirs-1)]

power_matrix[-300, 0:5, 0:5]

array([[1.        , 0.78262504, 0.3475298 , 0.79702984, 0.47046825],
       [0.78262504, 1.        , 0.49298815, 0.74560727, 0.53594114],
       [0.3475298 , 0.49298815, 1.        , 0.76439584, 0.54209184],
       [0.79702984, 0.74560727, 0.76439584, 1.        , 0.60889178],
       [0.47046825, 0.53594114, 0.54209184, 0.60889178, 1.        ]])

# Rough estimate for savings in bits (really need to truncate and not include some)

In [102]:
eigenvalues = [np.linalg.eigvals(power_matrix[i, :, :])for i in range(kbins)]

print(eigenvalues[2])
for i in range(kbins-300):
    print(np.sum(1/2*np.log2(eigenvalues[i])))

[8.76186013e+00 1.98522708e-01 4.00265959e-02 7.40922173e-03
 1.39812155e-03 5.24656623e-04 1.49813270e-04 7.25370135e-05
 3.59847341e-05 2.27763768e-07]
-61.96700371762052
-52.61035293595188
-47.291531866913076
-40.9987213963639
-38.30394713904842
-35.004171987728505
-33.02967348762414
-31.072090207731
-30.790954610597634
-29.49589580819765
-28.21805386997253
-27.050662492725394
-26.788441624382862
-25.60531367523319
-24.94967853316447
-24.77708501481129
-23.907103395157108
-23.311367118049702
-22.58884574088004
-22.39748723364491
-21.887169685668773
-21.798405623907914
-21.027847991857758
-21.063491663434704
-20.695735287993344
-20.178270304174603
-20.447697228686806
-20.020284606633545
-20.010468404364257
-19.255111508554062
-19.204331308903758
-19.119440120134758
-18.763698092176647
-18.382453496895504
-18.60422390651561
-18.375445058619626
-17.882402471101187
-17.971466249197558
-17.702463675362146
-17.422256755917324
-17.383679299344436
-17.364778611849317
-17.333204671728726
-16